In [1]:
import datetime
import numpy as np
import talib as ta
import alpaca_trade_api as tradeapi
import time
import sys
sys.path.insert(0, '../')
import pandas
from datetime import timedelta

In [2]:
from config_paper import paper_key_id, paper_secret_key
from cross_signal import *

In [3]:
#The url at the end is what makes a paper trading bot
base_url='https://paper-api.alpaca.markets'
api = tradeapi.REST(paper_key_id, paper_secret_key,base_url)

In [4]:
startdate = datetime.datetime.today().strftime("%Y-%m-%d")
startdate

'2020-05-22'

In [5]:
#make sure to import these packages
from pandas import DataFrame
import talib.abstract as tab
#abstract automatically extracts the columns and periods from the data itself, no input required



#feel free to add or subtract more indicators for any other strategy you want to use
def populateindicators(dataframe) -> DataFrame:
        #exponential moving averages
        
        dataframe['minusdi'] = tab.MINUS_DI(dataframe, timeperiod=14)
        dataframe['plusdi'] = tab.PLUS_DI(dataframe, timeperiod=14)
        #srsi used for finding buys at 71
        
        #dataframe['macd'],dataframe['macdsig'],dataframe['hist']
        
        dataframe['adx'] = tab.ADX(dataframe)
        
        dataframe['adxcross1'] =crossed_above(dataframe['adx'],17)
        dataframe['adxcross2'] =crossed_above(dataframe['adx'],23)
        dataframe['adxcross3'] =crossed_above(dataframe['adx'],31)
        
        dataframe['adxcross4'] =crossed_above(dataframe['adx'],23)
        dataframe['adxcross5'] =crossed_above(dataframe['adx'],50)
        
        # required for graphing

        return dataframe

In [6]:
account = api.get_account()
print(account.status)
account

ACTIVE


Account({   'account_blocked': False,
    'account_number': 'PA2P2YHF9VC8',
    'buying_power': '120000',
    'cash': '30000',
    'created_at': '2020-05-22T13:16:26.041985Z',
    'currency': 'USD',
    'daytrade_count': 0,
    'daytrading_buying_power': '120000',
    'equity': '30000',
    'id': 'b863af1e-1cbd-4782-9609-d82ad9d2e3f6',
    'initial_margin': '0',
    'last_equity': '30000',
    'last_maintenance_margin': '0',
    'long_market_value': '0',
    'maintenance_margin': '0',
    'multiplier': '4',
    'pattern_day_trader': False,
    'portfolio_value': '30000',
    'regt_buying_power': '60000',
    'short_market_value': '0',
    'shorting_enabled': True,
    'sma': '0',
    'status': 'ACTIVE',
    'trade_suspended_by_user': False,
    'trading_blocked': False,
    'transfers_blocked': False})

In [7]:
with open('SP500_2020Q2SYM.txt', 'r') as f:
    symbols= f.read().split('\n')

In [8]:
symbollist=pd.read_csv('nasdaq-100-index-05-06-2020.csv')['Symbol']

In [9]:
barTimeframe = "1Min" # 1Min, 5Min, 15Min, 1H, 1D
assetsToTrade = symbollist
#assetsToTrade = [x.upper() for x in symbols]
positionSizing = 0.05

# Tracks position in list of symbols to download 
assetListLen = len(assetsToTrade)
trade=True

#create dict of buyorder.id, submitted_at
startorder_dict_list=[]


In [10]:
#get starting order by order id, see datetime and get rid
#create corresponding orders

In [12]:
trade=True
while trade==True:
    #
    currenthour=time.localtime().tm_hour
    currentminute=time.localtime().tm_min
    currenttime=currenthour*100+currentminute
    currenttimestamp=api.get_clock().timestamp
            
            
    #Manage dict  
    if currenttime <=1230 and currenttime >=650: 
        orderlist=api.list_orders()
        expired_order_list=[]
        
        for i in range(len(startorder_dict_list)):
            if startorder_dict_list[i]['datetime']+datetime.timedelta(seconds=15)<currenttimestamp:
                expired_order_list.append(i)
                try:
                    api.cancel_order(startorder_dict_list[i]['id'])
                    print('cancelled '+startorder_dict_list[i]['symbol'] )
                except:('Fail cancellation')
        
        #remove expired orders after cancellation or expiration
        indexes = expired_order_list
        for index in sorted(indexes, reverse=True):
            del expired_order_list[index]
        
        
        
        
        
        for i in range(assetListLen) :
            try:
                symbol = assetsToTrade[i]

                returned_data = api.polygon.historic_agg_v2(symbol, 1, 'minute',_from=startdate, to=startdate).df
                    #api.get_barset(symbol,barTimeframe,limit=150).df
                data=populateindicators(returned_data)
            except:
                print('Issue with data '+ symbol)

        
            try:
                if data['adx'][-1] > 22 and \
            data['mfi'][-1] < 25 and \
            data['ultosc'][-1] < 25 and \
            data['rsi'][-1] < 33 and \
            data['minusdi'][-1] > 25 and \
            data['minusdi'][-1] > data['plusdi'][-1]:
                        buyord=True
                        assetlist=api.list_positions()
                        for asset in assetlist:
                            # if asset is in the list of positions we have already bought, then do not buy again
                            if asset.symbol == symbol:
                                buyord=False
                                print(asset.symbol,asset.qty)

                        #if asset is not already bought, then buy             
                        if buyord==True:
                            #cashBalance = account.daytrading_buying_power
                            #equity
                            cashBalance = float(account.buying_power)
                            targetPositionSize = int(cashBalance / (data['close'][-1] / positionSizing))
                            buyreturned = api.submit_order(symbol,targetPositionSize,"buy","limit","day",limit_price=data['close'][-1])
                            startorder_dict_list.append({'symbol':buyreturned.symbol,'id':buyreturned.id,'datetime':buyreturned.submitted_at,'side':buyreturned.side})
                            print('Buy ',buyreturned)

                    
            except:
               print("error with purchase ",symbol)
                
         

        
    
    time.sleep(3)
    #get current time and shut off trading after 12
     
    if currenthour >= 13: 
        trade=False
        print('Good work, Turning off')
    

    
    

tried 1 TTWO
tried 1 TTWO
tried 1 TTWO
tried 1 TTWO
tried 2 SPLK
tried 3 ADBE
tried 2 SPLK
tried 3 ADBE
tried 2 SPLK
tried 3 ADBE
tried 2 SPLK
tried 3 ADBE
tried 1 TMUS
tried 1 MAR
tried 1 TMUS
tried 1 MAR
tried 1 TMUS
tried 1 MAR
tried 1 TMUS
tried 3 LULU
Buy  Order({   'asset_class': 'us_equity',
    'asset_id': '31aa3cfa-8925-4cbc-98bc-74f411b7f17c',
    'canceled_at': None,
    'client_order_id': 'a62255ff-b3a3-491b-b069-99ab027eb64f',
    'created_at': '2020-05-22T13:56:03.848741Z',
    'expired_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'id': 'a20bcc73-1f7a-45a4-bb80-8478f5ef2b57',
    'legs': None,
    'limit_price': '271.04',
    'order_class': '',
    'order_type': 'limit',
    'qty': '22',
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': 'buy',
    'status': 'accepted',
    'stop_price': None,
    'submitted_at': '2020-05-22T13:56:03.812425Z',

cancelled LULU
cancelled ASML
cancelled GOOG
cancelled GOOGL
tried 1 AMAT
tried 1 ORLY
sell  Order({   'asset_class': 'us_equity',
    'asset_id': 'c73e2c28-af3f-4f5d-b550-a56b567ed47c',
    'canceled_at': None,
    'client_order_id': '81d8e467-973e-41bc-98a8-a87942680e2b',
    'created_at': '2020-05-22T14:07:22.646601Z',
    'expired_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'id': 'ad01f883-524a-4053-8a91-2662898b9990',
    'legs': None,
    'limit_price': '400.94',
    'order_class': '',
    'order_type': 'limit',
    'qty': '14',
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': 'sell',
    'status': 'accepted',
    'stop_price': None,
    'submitted_at': '2020-05-22T14:07:22.619033Z',
    'symbol': 'ORLY',
    'time_in_force': 'day',
    'type': 'limit',
    'updated_at': '2020-05-22T14:07:22.646601Z'})
tried 2 PAYX
tried 1 SIRI
SIRI -1154
cancelled

tried 3 NTAP
sell  Order({   'asset_class': 'us_equity',
    'asset_id': 'd37df28f-5646-48c8-99cc-4d0c9a5ec9ff',
    'canceled_at': None,
    'client_order_id': '62a65a7f-709c-4a1e-8ae7-5b557c2937c9',
    'created_at': '2020-05-22T14:09:14.36229Z',
    'expired_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'id': 'a230ae6a-adf4-4ee1-bde1-ed97dbede4f1',
    'legs': None,
    'limit_price': '44.15',
    'order_class': '',
    'order_type': 'limit',
    'qty': '135',
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': 'sell',
    'status': 'accepted',
    'stop_price': None,
    'submitted_at': '2020-05-22T14:09:14.346951Z',
    'symbol': 'NTAP',
    'time_in_force': 'day',
    'type': 'limit',
    'updated_at': '2020-05-22T14:09:14.36229Z'})
tried 1 SBUX
cancelled LULU
cancelled ASML
cancelled GOOG
cancelled GOOGL
cancelled ORLY
cancelled SIRI
cancelled ORLY
can

tried 2 DVA
cancelled LULU
cancelled ASML
cancelled GOOG
cancelled GOOGL
cancelled ORLY
cancelled SIRI
cancelled ORLY
cancelled ORLY
cancelled ORLY
cancelled ORLY
cancelled ORLY
cancelled ORLY
cancelled ORLY
cancelled EXPE
cancelled NTAP
cancelled PAYX
cancelled ADP
cancelled CMCSA
cancelled EXPE
cancelled PEP
cancelled VRSK
cancelled PEP
cancelled BIDU
cancelled DLTR
cancelled SBUX
tried 2 BIDU
sell  Order({   'asset_class': 'us_equity',
    'asset_id': '0ad15daa-ecfa-48e5-85e7-e9f5af988904',
    'canceled_at': None,
    'client_order_id': 'fbe86379-ff81-414d-9e91-235c7581294c',
    'created_at': '2020-05-22T14:14:46.58223Z',
    'expired_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'id': '4efa1a11-e269-4080-8857-580e8b5a7c20',
    'legs': None,
    'limit_price': '102.99',
    'order_class': '',
    'order_type': 'limit',
    'qty': '58',
    'replaced_at': None,
    'replaced_by': None,

tried 1 GOOGL
tried 1 MAR
tried 2 NFLX
sell  Order({   'asset_class': 'us_equity',
    'asset_id': 'bb2a26c0-4c77-4801-8afc-82e8142ac7b8',
    'canceled_at': None,
    'client_order_id': 'fba9fe18-0fb7-4c78-b19f-c297af28d042',
    'created_at': '2020-05-22T14:16:15.82377Z',
    'expired_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'id': '51c7887a-2941-4053-bcba-a3586d5bc2f4',
    'legs': None,
    'limit_price': '433.52',
    'order_class': '',
    'order_type': 'limit',
    'qty': '13',
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': 'sell',
    'status': 'accepted',
    'stop_price': None,
    'submitted_at': '2020-05-22T14:16:15.790621Z',
    'symbol': 'NFLX',
    'time_in_force': 'day',
    'type': 'limit',
    'updated_at': '2020-05-22T14:16:15.82377Z'})
tried 2 ORLY
tried 1 ROST
cancelled LULU
cancelled ASML
cancelled GOOG
cancelled GOOGL
cancelled

...

cancelled TTWO
cancelled VRSK
cancelled ALGN
cancelled CDNS
cancelled MSFT
cancelled TXN
cancelled ADI
cancelled ADSK
cancelled ALXN
cancelled CPRT
cancelled CSCO
cancelled ILMN
cancelled EXPE
cancelled FOX
cancelled IDXX
cancelled NVDA
cancelled MDLZ
cancelled SBUX
cancelled DXCM
cancelled INCY
cancelled MNST
cancelled CSX
cancelled ISRG
cancelled SPLK
cancelled EXC
cancelled LBTYK
cancelled DVA
cancelled ADI
cancelled FISV
cancelled CTAS
cancelled CSCO
cancelled FOXA
cancelled INTC
cancelled DVA
cancelled CTSH
cancelled EA
cancelled FOX
cancelled KLAC
cancelled UAL
cancelled AMZN
cancelled MCHP
cancelled AVGO
cancelled MAR
cancelled COST
cancelled SBUX
cancelled MXIM
cancelled AMAT
cancelled NTES
cancelled ALXN
cancelled MNST
cancelled GILD
cancelled ILMN
cancelled MSFT
cancelled NXPI
cancelled UAL
cancelled BIDU
cancelled BIIB
cancelled EXPE
cancelled GOOGL
cancelled MSFT
cancelled ADSK
cancelled BKNG
cancelled JD
cancelled MAR
cancelled ADP
cancelled BKNG
cancelled FOX
cancelled NV

cancelled ROST
cancelled ROST
cancelled SPLK
cancelled SPLK
cancelled SPLK
cancelled EXPE
cancelled EXPE
cancelled TCOM
cancelled MDLZ
cancelled CSCO
cancelled LBTYK
cancelled PCAR
cancelled FLIR
cancelled PEP
cancelled PEP
cancelled INTC
cancelled LBTYA
cancelled MDLZ
cancelled SGEN
cancelled WDC
cancelled TMUS
cancelled FAST
cancelled KLAC
cancelled NTES
cancelled LULU
cancelled TMUS
cancelled TTWO
cancelled DVA
cancelled INTU
cancelled NVDA
cancelled ULTA
cancelled CDW
cancelled DLTR
cancelled NFLX
cancelled LBTYK
cancelled VRSN
cancelled CHTR
cancelled WDAY
cancelled FOX
cancelled REGN
cancelled CMCSA
cancelled FAST
cancelled BMRN
cancelled WDC
cancelled ASML
cancelled CTXS
cancelled XLNX
cancelled ADI
cancelled MU
cancelled MXIM
cancelled NTAP
cancelled SWKS
cancelled WDC
cancelled MELI
cancelled BKNG
cancelled ILMN
cancelled BKNG
cancelled BKNG
cancelled BKNG
cancelled CTXS
cancelled NFLX
cancelled ULTA
cancelled ALGN
cancelled CSX
cancelled PEP
cancelled CSCO
cancelled CSGP
canc

tried 3 FB
tried 1 LBTYK
tried 1 LULU
sell  Order({   'asset_class': 'us_equity',
    'asset_id': '31aa3cfa-8925-4cbc-98bc-74f411b7f17c',
    'canceled_at': None,
    'client_order_id': '3f4b1778-f85e-44b1-b5f7-fbcbedcf5a66',
    'created_at': '2020-05-22T19:22:33.925024Z',
    'expired_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'id': '49de8f00-50a1-4898-b61d-607b88d22b00',
    'legs': None,
    'limit_price': '275.19',
    'order_class': '',
    'order_type': 'limit',
    'qty': '21',
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': 'sell',
    'status': 'accepted',
    'stop_price': None,
    'submitted_at': '2020-05-22T19:22:33.884194Z',
    'symbol': 'LULU',
    'time_in_force': 'day',
    'type': 'limit',
    'updated_at': '2020-05-22T19:22:33.925024Z'})
tried 2 MCHP
tried 2 SBUX
sell  Order({   'asset_class': 'us_equity',
    'asset_id': '0a7deac5

tried 2 CDW
CDW -56
tried 2 INTC
tried 1 MELI
Buy  Order({   'asset_class': 'us_equity',
    'asset_id': 'c85841d1-e05b-4f3c-9e9c-cc8fd3949fbb',
    'canceled_at': None,
    'client_order_id': '664e4e0d-8350-437f-8a01-17d6f27c2e5d',
    'created_at': '2020-05-22T19:25:58.542986Z',
    'expired_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'id': 'b8da5e4f-435c-45d7-9579-a1a46ebc7865',
    'legs': None,
    'limit_price': '842.29',
    'order_class': '',
    'order_type': 'limit',
    'qty': '7',
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': 'buy',
    'status': 'accepted',
    'stop_price': None,
    'submitted_at': '2020-05-22T19:25:58.489102Z',
    'symbol': 'MELI',
    'time_in_force': 'day',
    'type': 'limit',
    'updated_at': '2020-05-22T19:25:58.542986Z'})
tried 2 NXPI
tried 3 REGN
tried 1 SPLK
tried 3 UAL
tried 2 ULTA
cancelled LULU
cancelled A

cancelled BIIB
cancelled ILMN
cancelled NVDA
cancelled XLNX
cancelled ADP
cancelled GILD
cancelled TTWO
cancelled VRSN
cancelled SBUX
cancelled EXC
cancelled LULU
cancelled SBUX
cancelled SIRI
cancelled DVA
cancelled ADP
cancelled AVGO
cancelled MELI
tried 2 AMAT
tried 2 KHC
tried 2 PCAR
sell  Order({   'asset_class': 'us_equity',
    'asset_id': '29e34616-a842-4a30-908c-32eddd0c7e23',
    'canceled_at': None,
    'client_order_id': '9f562c5d-ead5-40b1-9fbd-78451c69fc70',
    'created_at': '2020-05-22T19:31:20.622247Z',
    'expired_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'id': '8ec2e3a9-fd25-4d0d-9cfa-61ae6edbc7d8',
    'legs': None,
    'limit_price': '68.18',
    'order_class': '',
    'order_type': 'limit',
    'qty': '88',
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': 'sell',
    'status': 'accepted',
    'stop_price': None,
    'submitted_at